In [1]:
#| default_exp testbed/visor

In [2]:
#| export
from __future__ import annotations


In [3]:
#| hide
# %reload_ext autoreload
# %autoreload 0


# Visor
> Simple visor of namespaces based on `ipywidgets`.


# Prologue

In [4]:
#| export
from collections import defaultdict
from typing import Any
from typing import TypeAlias

import ipywidgets as W
import traitlets as T
from IPython.display import clear_output
from IPython.display import display
from IPython.display import HTML
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from rich.console import Console


In [5]:
#| export

from pcleaner._testbed.testbed.helpers import _pops_
from pcleaner._testbed.testbed.helpers import cleanupwidgets


In [6]:
from fastcore.test import *  # type: ignore


# Helpers

In [7]:
# pretty print by default
# %load_ext rich

In [8]:
#| exporti
console = Console(width=104, tab_size=4, force_jupyter=True)
cprint = console.print


----
# ContextVisor


In [9]:
#| exporti

CtlT: TypeAlias = W.ValueWidget | W.fixed

NO_UI = W.DOMWidget(layout={'display': 'none'})
NO_UI.close()

#| export

UPDATE_SCRIPT = f"""
<script>
    var currentScript = document.currentScript;
    currentScript.parentNode.innerHTML = '';
    # console.log(currentScript);
    # debugger;
</script>
"""


In [10]:
#| export

class ContextVisor:
    ctx: Any
    values: dict[str, Any] = {}

    _ctxs: dict[str, ContextVisor] = {}
    _hdlrs: dict[str, ContextVisor] = {}
    _css: str = ''
    _ctl2name: dict[CtlT, str]
    _name2comp: dict[str, ContextVisor]
    _out: W.Output | None = None
    _w: W.DOMWidget | None = None
    _controls: dict[str, CtlT] | None = None
    _all_controls: dict[str, CtlT] | None = None
    _ui_cls: type[W.Box] = W.HBox
    _inited = False

    @property
    def w(self) -> W.DOMWidget:
        "Container (DOM)widget of this comp."
        if self._inited and self._w is None:
            self._w = self._setup_ui() or NO_UI
        return self._w  # type: ignore
    @property
    def out(self) -> W.Output:
        if self._inited and self._out is None:
            self._out = W.Output()
            self._out.clear_output(wait=True)
        return self._out  # type: ignore
    @property
    def controls(self) -> dict[str, CtlT]:
        if self._controls is None:
            self._controls = self.setup_controls() if self._inited else {}
        return self._controls
    @property
    def all_controls(self) -> dict[str, CtlT]:
        if self._all_controls is None:
            controls = {}
            if self._inited:
                for visor in self._ctxs.values():
                    controls.update(visor.all_controls)
                controls.update(self.controls)
            self._all_controls = controls
            self._ctl2name = {v:k for k,v in controls.items()}
        return self._all_controls  # type: ignore
    
    @property
    def all_values(self):
        """Values from first level comps, keyed by comp name.
        NOTE: will fail for nested contexts or more than one level.
        """
        return {**{k:v.values for k,v in (self.comps | {'self': self}).items()}}
    
    def _all_values(self):
        "Flattened values from all comps"
        all_values = {}
        for comp in [*self.comps.values(), self]: all_values.update(comp.values)
        return all_values

    @property
    def comps(self): return self._ctxs
    def comp(self, k: str) -> ContextVisor | None: return self._ctxs.get(k)
    def handler(self, k: str) -> ContextVisor | None: return self._hdlrs.get(k)
    
    @property
    def styler(self) -> W.HTML | str:
        if getattr(self, '_style', None) is None:
            stl = self.setup_style()
            if stl: 
                stl_id = 'stl-' + str(id(self))
                stl = f"<style id='{stl_id}'>{stl}</style>"
                self._style = W.HTML(stl)
            else:
                self._style = ''
        return self._style
    def setup_style(self):
        collate = [_.setup_style() for _ in self.comps.values()]
        if self._css: collate.append(self._css)
        return '\n'.join([_ for _ in collate if _])
    
    def update_output(self, **kwargs): 
        cprint(kwargs)
    
    def setup_controls(self) -> dict[str, CtlT]:
        widgets = [W.interactive.widget_from_abbrev(v) for k, v in self.values.items()]
        widgets = {k:W.fixed(v) if w is None else w for (k, v), w in zip(self.values.items(), widgets)}
        # return {k: W.Label(value=str(k)) for k,w in self.values.items()}
        return widgets
    
    def hide(self): 
        if (w := self.w) is not NO_UI: 
            w.layout.visibility = 'hidden'
    def show(self): 
        if (w := self.w) is not NO_UI: 
            w.layout.visibility = 'visible'

    def comps_ui(self):
        comps = []
        if self._inited: 
            for visor in self.comps.values():
                if (visor_ui := visor._setup_ui()) is not None:
                    comps.append(visor_ui)
        return comps


    def _setup_ui(self):
        if not self._inited: return
        w = self.setup_ui()
        if w is not None:
            w.add_class('context-visor')
            w.add_class(str(id(self)))
        return w
        
    def setup_ui(self) -> W.DOMWidget | None:
        """Get the container widget for this comp.
        This method should be the only one called when the comp is nested inside other comp.
        """
        uis = [*self.comps_ui(), *self.controls.values()]
        return self._ui_cls(uis) if uis else None

    def setup_display(self): 
        "Generates one time ui"
        if not self._inited: return
        if self._w is None:
            self._w = self._setup_ui()
        

    def _output(self, **kwargs):
        # group keys by comp
        collator = defaultdict(dict)
        for k,v in kwargs.items():
            if (comp := self.handler(k)) is not None:
                collator[comp][k] = v
            else:
                # keys w/out control assigned, considered internal state
                collator[self][k] = v
        # group comps by output
        outs = defaultdict(list)
        for comp, kw in collator.items():
            outs[comp.out].append((comp, kw))
        for out, g in outs.items():
            show_inline_matplotlib_plots()
            with out:
                clear_output(wait=True)
                for comp, kw in g:
                    comp.update_output(**kw)
                show_inline_matplotlib_plots()

    def _observe(self, change):
        control_name = self._ctl2name[change['owner']]
        kwargs = {control_name: change['new']}
        updated = self._update(**kwargs)
        self._output(**updated)
    def setup_ux(self): pass
    def _setup_ux(self): 
        for visor in self.comps.values():
            visor._setup_ux()
        self.setup_ux()
    def interactive_output(self):
        controls = self.all_controls
        all_values = self._all_values()
        for k,w in controls.items():
            if k in all_values:
                w.observe(self._observe, 'value')
    
    def display(self, **kwargs): 
        if not self._inited: return
        if self._w is None:
            self.setup_display()
            self.interactive_output()
            self._update(**(self.values | kwargs))
            all_values = self._all_values()
            self._hdlrs = {k:self._hdlrs.get(k, self) for k in all_values}
            self._output(**all_values)
            # ux final touches once everything (including outputs) is setup
            # for visor in [*self.comps.values(), self]:
            #     visor.setup_ux()
            self._setup_ux()
            stl = self.styler
            ui: list = [stl] if stl else []
            if (w := self.w) is not NO_UI:
                ui.append(w)
            for comp in [*self.comps.values(), self]:
                if comp._out is not None:
                    ui.append(comp._out)
            self._final = W.VBox(ui)
            self._display_handle = display(self._final, display_id=str(id(self)))
        else:
            self.update(**kwargs)
    def _ipython_display_(self): self.display()

    def _update(self, update_value: bool=True, **kwargs):
        updated = {}
        for visor in self.comps.values():
            updated.update(visor._update(update_value=update_value, **kwargs))
        values = self.values
        my_vals = _pops_(kwargs, self.values.keys())
        for k,v in my_vals.items():
            if v is not None and v != values[k]:
                if update_value: 
                    values[k] = v
                updated[k] = v
        return updated
    def update(self, **kwargs):
        updated = self._update(update_value=False, **kwargs)
        controls = self.all_controls
        for k,v in updated.items():
            if k in controls:
                if hasattr((ctl := controls[k]), 'value'):
                    ctl.value = v  # will trigger update (self._observe)
            elif k in (vv := self._name2comp):
                # update manually
                comp = vv[k]
                if v != comp.values[k]:
                    comp.values[k] = v
                    self._output(**{k:v})
    

    def close(self):
        controls = self.all_controls
        for w in controls.values():
            try: w.unobserve(self._observe, 'value')
            except: pass
            if isinstance(w, W.Widget):
                w.close()
        for visor in self._ctxs.values():
            if w := getattr(visor, '_w', None): w.close()
            if visor._out is not self._out:
                if o := getattr(visor, '_out', None): o.close()
            visor.close()
        if w := getattr(self, '_w', None): w.close()
        if o := getattr(self, '_out', None): o.close()
        if f := self._final: f.close()
        if self._display_handle is not None:
            self._display_handle.update(HTML(UPDATE_SCRIPT))


    def __del__(self): 
        self.close()

    def __init__(self, 
            ctx: Any, 
            values: dict[str, Any], 
            out: W.Output | None = None,
            ctxs: dict[str, ContextVisor] | None = None,
            hdlrs: dict[str, ContextVisor] | None = None,
            css: str | None = None,
        ):
        # Only setup some state. Controls, values and containers will be setup only when explicitly displayed
        self._display_handle = None
        self._final = None
        self.ctx = ctx
        self.values = values or {}
        self._out = out
        self._ctxs = comps = ctxs or {}
        self._hdlrs = hdlrs or {}
        if css is not None:
            self._css = css
        self._name2comp = name2comp = {}
        for n,vv in self.all_values.items():
            comp = comps.get(n, self)
            for k in vv:
                name2comp[k] = comp
        self._inited = True


In [11]:
cleanupwidgets('test_visor')

test_visor = ContextVisor(None, {'a': 1})
test_visor


In [12]:
test_eq(test_visor.values, {'a': 1})
test_visor.update(a='2')
test_eq(test_visor.values, {'a': 2})


In [13]:
cleanupwidgets('test_visor')
test_eq(test_visor.w.comm, None)


In [14]:
cleanupwidgets('vsr1')

vsr1 = ContextVisor(
    None, {'a': 3}, 
    ctxs={'vsr2': ContextVisor(None, {'b': 'bbb'})}, 
)
test_eq(vsr1.values, {'a': 3})
test_eq(vsr1.all_values, {'vsr2': {'b': 'bbb'}, 'self': {'a': 3}})
vsr1.display()



In [15]:
vsr1.update(a=2)
test_eq(vsr1.all_values, {'vsr2': {'b': 'bbb'}, 'self': {'a': 2}})


In [16]:
vsr1.update(a=0, b='ccc')
test_eq(vsr1.all_values, {'vsr2': {'b': 'ccc'}, 'self': {'a': 0}})


In [17]:
cleanupwidgets('vsr2')

vsr3 = ContextVisor(None, {'b': 'bbb'})
vsr2 = ContextVisor(
    None, {'a': 3}, 
    ctxs={'vsr3': vsr3}, 
    hdlrs={'b': vsr3}
)
vsr2.display()
test_eq(vsr2.values, {'a': 3})
test_eq(vsr2.all_values, {'vsr3': {'b': 'bbb'}, 'self': {'a': 3}})


In [18]:
vsr2.update(a=1, b='ccc')
test_eq(vsr2.values, {'a': 1})
test_eq(vsr2.all_values, {'vsr3': {'b': 'ccc'}, 'self': {'a': 1}})


In [19]:
class Vsr1(ContextVisor):
    _css = """
    .vsr1 {
        border: 1px solid red;
    }
    """
    def setup_ui(self) -> W.DOMWidget | None:
        w = super().setup_ui()
        if w is not None:
            w.add_class('vsr1')
        return w

class Vsr2(ContextVisor):
    _css = """
    .vsr2 {
        border: 1px solid green;
    }
    """
    def setup_ui(self) -> W.DOMWidget | None:
        uis = [*self.comps_ui(), *self.controls.values()]
        w = W.HBox(uis) if uis else None
        if w is not None:
            w.add_class('vsr2')
        return w

vsr1 = Vsr1(
    None, {'a': 3}, 
    ctxs={'vsr2': Vsr2(None, {'b': 'bbb'})}, 
    css = """
    .vsr1 {
        border: 1px solid red;
    }
    """
)
test_eq(vsr1.values, {'a': 3})
test_eq(vsr1.all_values, {'vsr2': {'b': 'bbb'}, 'self': {'a': 3}})
vsr1.display()


In [20]:
cleanupwidgets('vsr1')


# Spinner

In [21]:
#| exporti

spinner_css = """
    .wrapper-spinner {
        overflow: hidden;
        width: fit-content;
        height: fit-content;
    }
    
    .loading-spinner {
        display: flex;
        align-items: center;
        justify-content: center;
        border: 1px solid white;
        border-radius: 50%;
    }
    
    .spinner {
        border: |border_width|px solid rgba(128,128,128,.5);
        border-radius: 50%;
        border-left-color: red;
        animation: spin 1s infinite linear;
    }
    
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
"""


In [22]:
# display(HTML(f"<style>{spinner_css.replace('|border_width|', '4')}</style>"))


In [23]:
# def loading_spinner(size=36, border_width=4):
#     bw = border_width*2
#     html: str = f'''
# <div class="loading-spinner" style="width: {size + bw}px; height: {size + bw}px;">
#     <div class="spinner" style="width: {size}px; height: {size}px;"></div>
# </div>
# '''
#     return html

In [24]:
# cleanupwidgets('spinner')

# spinner = W.HTML(loading_spinner(24), 
#     # layout={
#     #     'overflow': 'hidden', 
#     #     'width': 'fit-content', 
#     #     'height': 'fit-content',
#     #     'border': '1px solid green'
#     #     }
#         )
# spinner.add_class('wrapper-spinner')
# spinner


In [25]:
# spinner.layout.display = 'none'


In [26]:
# spinner.layout.display = 'block'


In [27]:
# cleanupwidgets('spinner')

In [28]:
#| export

class Spinner(ContextVisor):
    ctx: T.HasTraits

    def loading_spinner(self, size=36, border_width=4):
        bw = border_width*2
        html: str = f'''
<div class="loading-spinner" style="width: {size + bw}px; height: {size + bw}px;">
    <div class="spinner" style="width: {size}px; height: {size}px;"></div>
</div>
    '''
        return html
    
    def setup_controls(self):
        spinner = W.HTML(self.loading_spinner(self.size, self.border_width))
        spinner.add_class('wrapper-spinner')
        return {'spinner': spinner}

    @property
    def spinner(self) -> W.HTML: return self.controls['spinner']  # type: ignore

    def hide(self): self.spinner.layout.display = 'none'
    def show(self): self.spinner.layout.display = 'block'

    def setup_ux(self):
        source = (self.ctx, '_running')
        target = (self.spinner.layout, 'display')
        self._link = T.dlink(source, target, lambda x: 'block' if x else 'none')

    def close(self):
        if l := getattr(self, '_link', None): l.unlink()
        super().close()

    def __init__(self, 
            ctx: T.HasTraits,
            size: int = 24,
            border_width: int = 4,
            **kwargs
        ):
        self.size = size
        self.border_width = border_width
        self._link = None
        super().__init__(ctx, {}, css=spinner_css.replace('|border_width|', str(border_width)), **kwargs)


In [29]:
cleanupwidgets('spinner')

class _Test(T.HasTraits):
    _running = T.Bool(True)

test = _Test()
spinner = Spinner(test)
spinner.display()


In [30]:
# spinner.hide()
test._running = False


In [31]:
test._running = True


In [32]:
cleanupwidgets('spinner')
test_eq(test._trait_notifiers, {'_running': {'change': []}})


In [33]:
cleanupwidgets('vsr3')

vsr3 = Vsr1(
    None, {'a': 3}, 
    ctxs={'vsr2': Vsr2(None, {'b': 'bbb'}, ctxs={'spinner': Spinner(test, 20, 3)})}, 
    css = """
    .vsr3 {
        border: 1px solid red;
    }
    """
)
test_eq(vsr3.values, {'a': 3})
test_eq(vsr3.all_values, {'vsr2': {'b': 'bbb'}, 'self': {'a': 3}})
vsr3.display()


In [34]:
test._running = False


In [35]:
test._running = True


In [36]:
cleanupwidgets('vsr3')

# Colophon
----


In [37]:
import fastcore.all as FC
from nbdev.export import nb_export


In [38]:
if FC.IN_NOTEBOOK:
    nb_export('visor.ipynb', '..')
